In [1]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

In [2]:
mp_pose = mp.solutions.pose

def extract_landmarks_from_video(video_path):
    cap = cv2.VideoCapture(video_path)
    pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

    landmark_rows = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Procesar frame con MediaPipe
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame_rgb)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            row = {}
            for i, lm in enumerate(landmarks):
                row[f'x{i}'] = lm.x
                row[f'y{i}'] = lm.y
                row[f'v{i}'] = lm.visibility
            # Otras columnas necesarias (puedes asignar una clase dummy si no la tienes)
            row['class_name'] = 'unknown'
            row['label'] = -1
            landmark_rows.append(row)

    cap.release()
    pose.close()

    # Convertir a DataFrame
    df_landmarks = pd.DataFrame(landmark_rows)
    return df_landmarks

# Ejemplo
video_path = "../data/validation/video5.mp4"
df_new_video = extract_landmarks_from_video(video_path)
print("✅ Landmarks extraídos:", df_new_video.shape)


✅ Landmarks extraídos: (195, 101)


In [13]:
# Calcular visibilidad promedio
vis_cols = [f'v{i}' for i in range(33)]
df_new_video['v_mean'] = df_new_video[vis_cols].mean(axis=1)

# Filtrar por visibilidad
df_new_video = df_new_video[df_new_video['v_mean'] >= 0.3]

# Filtrar coordenadas fuera de rango
coord_cols = [col for col in df_new_video.columns if col.startswith(('x', 'y'))]
df_new_video = df_new_video[(df_new_video[coord_cols] >= 0.0).all(axis=1) & (df_new_video[coord_cols] <= 1.0).all(axis=1)]

print("✅ Dataset limpio de landmarks:", df_new_video.shape)


✅ Dataset limpio de landmarks: (195, 102)


In [14]:
def calc_angle(a, b, c):
    """
    Calcula el ángulo entre tres puntos: a (proximal), b (vértice), c (distal)
    Devuelve el ángulo en grados.
    """
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)


def extract_features_from_row(row):
    features = {}

    hip_left = [row['x23'], row['y23']]
    knee_left = [row['x25'], row['y25']]
    ankle_left = [row['x27'], row['y27']]

    hip_right = [row['x24'], row['y24']]
    knee_right = [row['x26'], row['y26']]
    ankle_right = [row['x28'], row['y28']]

    shoulder_left = [row['x11'], row['y11']]
    shoulder_right = [row['x12'], row['y12']]

    features['angle_knee_left'] = calc_angle(hip_left, knee_left, ankle_left)
    features['angle_knee_right'] = calc_angle(hip_right, knee_right, ankle_right)
    features['angle_hip_left'] = calc_angle(shoulder_left, hip_left, knee_left)
    features['angle_hip_right'] = calc_angle(shoulder_right, hip_right, knee_right)

    trunk_vector = np.array(shoulder_right) + np.array(shoulder_left) - np.array(hip_right) - np.array(hip_left)
    features['trunk_inclination'] = np.arctan2(trunk_vector[1], trunk_vector[0]) * 180 / np.pi

    features['shoulder_dist'] = np.linalg.norm(np.array(shoulder_left) - np.array(shoulder_right))
    features['hip_dist'] = np.linalg.norm(np.array(hip_left) - np.array(hip_right))

    return features

# Generar features
feature_rows = []
for idx, row in df_new_video.iterrows():
    feats = extract_features_from_row(row)
    feature_rows.append(feats)

df_new_features = pd.DataFrame(feature_rows)
print("✅ Features generados:", df_new_features.shape)


✅ Features generados: (195, 7)


In [15]:
# Cargar modelo ya entrenado
model = tf.keras.models.load_model("./models/movimiento_classifier.h5")
print("✅ Modelo cargado correctamente")

# Predecir
# Supongamos que tienes un nuevo DataFrame de features llamado `df_new_features`
# y quieres hacer la predicción de clases

predictions = model.predict(df_new_features)
# La salida es un array de probabilidades por clase
predicted_labels = predictions.argmax(axis=1)

print("Etiquetas predichas:", predicted_labels)

predicted_class = np.bincount(predicted_labels).argmax()
print("Clase global del video:", predicted_class)

if predicted_class == 0:
    print("CAMINAS HACIA ADELANTE")
elif predicted_class == 1:
    print("CAMINAS HACIA ATRÁS")
elif predicted_class == 2:
    print("TE LEVANTAS")
elif predicted_class == 3:
    print("TE SIENTAS")
elif predicted_class == 4:
    print("DAS UNA VUELTA")
else:
    print("El video no corresponde a ninguna clase conocida.")


✅ Modelo cargado correctamente
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Etiquetas predichas: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0]
Clase global del video: 0
CAMINAS HACIA ADELANTE


In [ ]:
import numpy as np


Clase global del video: 0


In [ ]:
unique, counts = np.unique(predicted_labels, return_counts=True)
for u, c in zip(unique, counts):
    print(f"Clase {u}: {c} frames")